<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#KMeans-Clustering" data-toc-modified-id="KMeans-Clustering-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>KMeans Clustering</a></span><ul class="toc-item"><li><span><a href="#Description" data-toc-modified-id="Description-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Description</a></span></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Cluster" data-toc-modified-id="Cluster-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Cluster</a></span></li></ul></li><li><span><a href="#Samples-per-Fund--&gt;-weights-per-fund" data-toc-modified-id="Samples-per-Fund-->-weights-per-fund-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Samples per Fund -&gt; weights per fund</a></span></li><li><span><a href="#Visualize" data-toc-modified-id="Visualize-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Visualize</a></span></li></ul></div>

# KMeans Clustering

In [ ]:
t0 = time() 
tfidf = tfidf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))

## Description

- Try different clusterings 

In [ ]:
import feather
import multiprocessing

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scipy import sparse

import matplotlib.pyplot as plt
import matplotlib.cm as cm

from sklearn.cluster import KMeans, MiniBatchKMeans, DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import Normalizer, MaxAbsScaler

import hdbscan

## Load Data

In [ ]:
path = '../data/processed/EDY/returns_s.feather'
returns = feather.read_dataframe(path)
returns.shape

In [ ]:
path = '../data/processed/EDY/holdings_summary_s.feather'
summary = feather.read_dataframe(path)
summary.shape

In [ ]:
path = '../data/processed/EDY/holdings_s.npz'
holdings_t = sparse.load_npz(path)
holdings_t.shape

In [ ]:
path = '../data/processed/EDY/holdings_s.npz'
holdings = sparse.load_npz(path)
holdings.shape

## Cluster

In [ ]:
pd.DataFrame(holdings.sum(0)).plot()

pd.DataFrame(holdings.sum(1)).plot()

In [ ]:
print(multiprocessing.get_start_method())

In [ ]:
multiprocessing.set_start_method('forkserver', force = True)
print(multiprocessing.get_start_method())
print("Changed start method")

In [ ]:
holdings = holdings_t[:]

In [ ]:
%%time
# ('ward', 'average', 'complete', 'single'):

clusterer = AgglomerativeClustering(linkage='ward', n_clusters=4)

fit = clusterer.fit(holdings.toarray())

In [ ]:
plt.hist(fit.labels_)

In [ ]:
%%time
print('Start kMeans...')
ç = KMeans(n_clusters = 4,
                verbose = True,
                n_init = 8, # Number of runs
                n_jobs= -1,
                random_state=0).fit(holdings)

In [ ]:
lables = pd.DataFrame(kmeans.labels_)

In [ ]:
lables.sample(10)

In [ ]:
summary = summary.loc[:1000,:]
summary['cluster'] = fit.labels_

pd.crosstab(summary['crsp_obj_cd'],summary['cluster'])# Samples per Fund -> weights per fund

In [ ]:
index = pd.DataFrame(np.arange(summary.shape[0]))

index = index.groupby(summary['port_no']).tail(20)

index = index.values.T.flatten()

summary = summary.loc[index,:]
holdings = holdings[index]
holdings.shape

#### Sonstiges

In [ ]:
%%time

#X = holdings
n_clusters = [2]

clusters = []
results = []

for n_clusters in n_clusters:
    clusterer = KMeans(n_clusters = 5,
                    verbose = True,
                    n_init = 10, # Number of runs
                    n_jobs= -1,
                    random_state=0)
    
    
    cluster_fit = clusterer.fit(X)
#    clusters.append(n_clusters)
#    results.append(cluster_fit.inertia_)    
    
    
    
    cluster_labels = cluster_fit.labels_
    
    # Create a subplot with 1 row and 2 columns
    fig, ax1 = plt.subplots()
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, X.shape[0] + (n_clusters + 1) * 10])
    
    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample

    # TODO can pool.map be implemented here?
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

plt.show()

# Samples per Fund -> weights per fund

In [ ]:
index = pd.DataFrame(np.arange(summary.shape[0]))

In [ ]:
index = index.groupby(summary['port_no']).tail(20)

In [ ]:
index = index.values.T.flatten()

In [ ]:
summary = summary.loc[index,:]
holdings = holdings[index]
holdings.shape

#### Sonstiges

In [ ]:
result_mat = pd.DataFrame({'Clusters': clusters, 'Inertia':results})

In [ ]:
plt.plot(result_mat['Clusters'],result_mat['Inertia'])

# Visualize

In [ ]:
MiniBatchKMeans?

In [ ]:
%%time
holdings = holdings[:3000]
X = holdings
n_clusters = [4]

clusters = []
results = []



for n_clusters in n_clusters:
    clusterer = DBSCAN(
                        eps=0.5, 
                        min_samples=5, 
                        metric='euclidean', 
                        metric_params=None, 
                        leaf_size=50, 
                        p=None, 
                        n_jobs=-1)
    
    
    cluster_fit = clusterer.fit(holdings)
#    clusters.append(n_clusters)
#    results.append(cluster_fit.inertia_)    
    
    
    
    cluster_labels = cluster_fit.labels_
    
    # Create a subplot with 1 row and 2 columns
    fig, ax1 = plt.subplots()
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, X.shape[0] + (n_clusters + 1) * 10])
    
    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample

    # TODO can pool.map be implemented here?
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

plt.show()